# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.4,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.01), (0.0,0,-0.001), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True, parent="floor")
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

 * Environment: production
Please create a subscriber to the marker
   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

In [7]:
# gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole_col", link_name="indy0_tcp",

## add wp

In [8]:
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.08,0.08,0.01), (0.1,0.2,0.0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="floor")
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.08,0.08,0.01), (0.1,-0.2,0.0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="floor")
line1 = gscene.create_safe(GEOTYPE.BOX, "wline1", "base_link", (0.01,0.5,1e-6), (0.1,0,0.005), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="floor")

wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.08,0.08,0.01), (0.0,0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="floor")
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.08,0.08,0.01), (0.0,-0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="floor")
line2 = gscene.create_safe(GEOTYPE.BOX, "wline2", "base_link", (0.01,0.5,1e-6), (0.0,0,0.005), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="floor")

wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.08,0.08,0.01), (-0.1,0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="floor")
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.08,0.08,0.01), (-0.1,-0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="floor")
line3 = gscene.create_safe(GEOTYPE.BOX, "wline3", "base_link", (0.01,0.5,1e-6), (-0.1,0,0.005), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="floor")

## add brush

In [9]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="indy0_tcp", dims=(0.1,0.07,0.02), 
                   center=(0.0,0.0,0.14), rpy=(0,np.pi,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="indy0_tcp", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp", dims=(0.1,0.06,0.028), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")

## add box

In [10]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.4,-0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

# gbox3 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box3", link_name="base_link", dims=(0.05,0.05,0.05), 
#                    center=(0.3,-0.2,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## Register binders

In [11]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool, SweepFramer, FixtureSlot

In [12]:
# gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
#                  dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [13]:
# pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=FixtureSlot, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", rname="panda1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="brush_face", gname="brush_face", rname="indy0", _type=SweepFramer, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [14]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [15]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
# brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
#                              action_points_dict = {"handle": FixturePoint("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
#                                                    "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [16]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_object(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)
# box3= pscene.create_object(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [17]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [18]:
sweep1 = pscene.create_object(oname="sweep1", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp11": SweepFrame("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepFrame("wp12", wp12, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line1)
sweep2 = pscene.create_object(oname="sweep2", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp21": SweepFrame("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepFrame("wp22", wp22, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line2)
sweep3 = pscene.create_object(oname="sweep3", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp31": SweepFrame("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepFrame("wp32", wp32, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line3)

### planners

In [19]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [20]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker

gcheck = GraspChecker(pscene, 
                      end_link_couple_dict= {
                          "indy0_tcp": ["indy0_tcp", "indy0_link6"],
                          "panda1_hand": ["panda1_hand", "panda1_link6"],
                          "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [21]:
mplan.motion_filters = checkers_all

In [22]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [23]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('floor', 'floor', 0, 0, 0)


In [24]:
pscene.subject_name_list

['box1', 'box2', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [25]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler

In [26]:

##
# @class NodeSampler
# @biref uniformly sample nodes by decaying probability of sampled node
class GrowingSampler:
    def __init__(self, log2_decay=1):
        self.log2_decay = log2_decay

    def init(self, tplan, multiprocess_manager):
        self.node_dict = tplan.node_dict
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.log2_prob_dict = multiprocess_manager.dict()
            self.prob_lock = multiprocess_manager.Lock()
        else:
            self.log2_prob_dict = dict()
            self.prob_lock = DummyBlock()

    def __call__(self, nodes):
        nodes = list(nodes)
        for node in nodes:
            if node not in self.log2_prob_dict:
                self.log2_prob_dict[node] = 1.0
        probs = [np.exp2(self.log2_prob_dict[node]) for node in nodes]
        sumprobs = np.sum(probs)
        if sumprobs == 0:
            print("probability saturated")
            probs = np.ones_like(probs)/len(probs)
        else:
            probs = np.divide(probs, sumprobs)
        i_node = np.random.choice(len(nodes), p=probs)
        node = nodes[i_node]
        with self.prob_lock:
            self.log2_prob_dict[node] = self.log2_prob_dict[node] - self.log2_decay
        return node

    
##
# @class NodeSampler
# @biref uniformly sample nodes by decaying probability of sampled node
class PenaltyNodeSampler:
    def __init__(self, log2_decay=1, log2_count_decay=1):
        self.log2_decay = log2_decay
        self.log2_count_decay = log2_count_decay

    def init(self, tplan, multiprocess_manager):
        self.node_dict = tplan.node_dict
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.log2_prob_dict = multiprocess_manager.dict()
            self.log2_count_dict = multiprocess_manager.dict()
            self.prob_lock = multiprocess_manager.Lock()
        else:
            self.log2_prob_dict = dict()
            self.log2_count_dict = dict()
            self.prob_lock = DummyBlock()

        for k in self.node_dict.keys():
            self.log2_prob_dict[k] = 1.0
            self.log2_count_dict[k] = 0.0

    def __call__(self, nodes):
        nodes = list(nodes)
        probs = [np.exp2(self.log2_prob_dict[node]+self.log2_count_dict[node]) for node in nodes]
        sumprobs = np.sum(probs)
        if sumprobs == 0:
            print("probability saturated")
            probs = np.ones_like(probs)/len(probs)
        else:
            probs = np.divide(probs, sumprobs)
        i_node = np.random.choice(len(nodes), p=probs)
        node = nodes[i_node]
        with self.prob_lock:
            self.log2_prob_dict[node] = self.log2_prob_dict[node] - self.log2_decay
        return node
    
    def update(self, snode_src, snode_new, connection_result):
        if connection_result:
            self.log2_count_dict[snode_new.state.node] = self.log2_count_dict[snode_new.state.node] - self.log2_count_decay
            

In [27]:
tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

In [28]:
class BindingSampler:
    def __init__(self, gamma_decay=0.8):
        self.gamma_decay = gamma_decay

    def init(self, tplan, multiprocess_manager):
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.prob_dict = multiprocess_manager.dict()
            self.prob_lock = multiprocess_manager.Lock()
        else:
            self.prob_dict = dict()
            self.prob_lock = DummyBlock()

    def __call__(self, nodes):
        nodes = list(nodes)
        probs = [self.prob_dict[node] for node in nodes]
        probs = np.divide(probs, np.sum(probs))
        i_node = np.random.choice(len(nodes), p=probs)
        node = nodes[i_node]
        with self.prob_lock:
            self.prob_dict[node] = self.prob_dict[node] * self.gamma_decay
        return node

# CustomRule

In [29]:
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [30]:
class CustomRule:
    def __init__(self, pscene):
        self.pscene = pscene
        self.chain_dict = pscene.get_robot_chain_dict()
        
    def init(self, tplan, multiprocess_manager):
        self.multiprocess_manager = multiprocess_manager
        if multiprocess_manager is not None:
            self.call_count_dict = multiprocess_manager.dict()
            self.count_lock = multiprocess_manager.Lock()
        else:
            self.call_count_dict = dict()
            self.count_lock = DummyBlock()
            
        no_enter_sidxes = [stype==SweepLineTask for stype in enumerate(self.pscene.subject_type_list)]
        no_enter_initials = [tplan.initial_state.node[sidx] for sidx in no_enter_sidxes]
        self.sorted_unstop_keys = sorted(tplan.unstoppable_terminals.keys())
            
        self.node_dict={}
        self.enter_dict={}
        for node, leafs in tplan.node_dict.items():
            self.node_dict[node] = deepcopy(leafs)
            if not all([node[k] in terms for k, terms in tplan.unstoppable_terminals.items()]):
                tplan.node_dict[node] = set([]) ## unstoppable node change will be reserved by this custom rule
            else:
                # entering to unstoppable terminal is controlled by this rule
                self.enter_dict[node] = set([leaf for leaf in leafs 
                                             if any([leaf[k] not in terms for k, terms in tplan.unstoppable_terminals.items()])
                                             and all([leaf[k]>=leaf[k+1] for k in self.sorted_unstop_keys[:-1]])]) # the task is done in order
                tplan.node_dict[node] = set([leaf for leaf in leafs 
                                             if all([leaf[k] in terms for k, terms in tplan.unstoppable_terminals.items()])
                                            ])

        self.node_parent_dict={}
        for node, parents in tplan.node_parent_dict.items():
            self.node_parent_dict[node] = deepcopy(parents)
            tplan.node_parent_dict[node] = parents = set(
                [parent for parent in parents ## unstoppable node change will be reserved by this custom rule
                 if all([parent[k] in terms for k, terms in tplan.unstoppable_terminals.items()])])
        
    def __call__(self, tplan, snode_src, snode_new, connection_result):
        print("CustomRule call")
        stack_res = False
        stack_items = []
        if connection_result and snode_new.state.node in self.enter_dict:
            with tplan.snode_dict_lock:
                nb_nodes = list(tplan.neighbor_nodes.keys())
                for nb_node in nb_nodes:
                    for ukey in self.sorted_unstop_keys:
                        if nb_node[ukey] < snode_new.state.node[ukey]:
                            del tplan.neighbor_nodes[nb_node]
                            break # task is done in order
        
        if connection_result and snode_new.state.node in self.enter_dict:
            stack_res, stack_items = True, list(self.enter_dict[snode_new.state.node])
            
        if snode_src is not None:
            with self.count_lock:
                if snode_src.idx in self.call_count_dict:
                    self.call_count_dict[snode_src.idx] = self.call_count_dict[snode_src.idx] + [snode_new.state.node]
                else:
                    self.call_count_dict[snode_src.idx] = [snode_new.state.node]
            
            node_src = snode_src.state.node
            node_new = snode_new.state.node
            diff_sidxes = np.where([ ntem_s != ntem_g for ntem_s, ntem_g in zip(node_src, node_new)])[0]
            print("{}->{} , diff: {}".format(node_src, node_new, diff_sidxes))
            if len(diff_sidxes)==0:
                return stack_res, stack_items
            diff_sidx = diff_sidxes[0]
            diff_sname = self.pscene.subject_name_list[diff_sidx]
            diff_subject = self.pscene.subject_dict[diff_sname]
            
            if isinstance(diff_subject, SweepLineTask):
                print("Rule for SweepLineTask")
                with tplan.snode_dict_lock:
                    if node_src[diff_sidx] not in tplan.unstoppable_terminals[diff_sidx]: # from intermediate wp -> remove access to them
                        snode_list = tplan.node_snode_dict[node_src]
                        if snode_src.idx in snode_list:
                            print("Removing {} from {}: {} not in {}".format(snode_src.idx, snode_list, node_src[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
                            print(snode_list)
                            snode_list.remove(snode_src.idx)
                            tplan.node_snode_dict[node_src] = snode_list
                            print(tplan.node_snode_dict[node_src])
                        if len(snode_list) == 0 and node_new in tplan.neighbor_nodes:
                            del tplan.neighbor_nodes[node_new]
                    if connection_result:
                        if node_new[diff_sidx] in tplan.unstoppable_terminals[diff_sidx]: ## in terminal condition
                            print("Check home: {} in {}".format(node_new[diff_sidx], tplan.unstoppable_terminals[diff_sidx]))
                            link_name = self.pscene.gscene.NAME_DICT[snode_new.state.binding_state[diff_sidx][-1]].link_name
                            rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                                            link_name in chain_vals['link_names']]
                            if len(rname_candis)==0:
                                print("no robot candis")
                            else:
                                print("reserve homing = {}".format(snode_new.idx))
                                newstate = snode_new.state.copy(self.pscene)
                                jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                                newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                                return True, [newstate] + stack_items
                        else:## not in terminal condition
                            print("Reserve next wp from {}".format(snode_new.idx))
                            snode_list = tplan.node_snode_dict[node_new]
                            if snode_new.idx in snode_list:
                                print("Removing {} from {} as next wp is reserved".format(snode_new.idx, snode_list))
                                snode_list.remove(snode_new.idx)
                                tplan.node_snode_dict[node_new] = snode_list
                            next_node_candis = list(self.node_dict[node_new])
                            assert len(next_node_candis) == 1, "non-terminal sweep task should have only 1 leaf ({}) {}-{}".format(diff_sidx, node_new, next_node_candis)
                            return True, next_node_candis + stack_items
                return stack_res, stack_items
            
            if isinstance(diff_subject, AbstractObject):
                print("Rule for AbstractObject")
                if connection_result:
                    link_name1 = snode_src.state.state_param[diff_sname][0]
                    link_name2 = snode_new.state.state_param[diff_sname][0]
                    rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                                   link_name1 in chain_vals['link_names'] or link_name2 in chain_vals['link_names']]
                    if len(rname_candis)==0:
                        print("no robot candis")
                        return stack_res, stack_items
                    else:
                        print("try homing")
                        newstate = snode_new.state.copy(self.pscene)
                        jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                        newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                        return True, [newstate] + stack_items
        return stack_res, stack_items

In [31]:
tplan.custom_rule = CustomRule(pscene)

In [32]:
gtimer.reset()
goal_nodes = [("floor","floor", 2, 2, 2)]
# goal_nodes = [("floor","floor","floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=5)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
CustomRule call
make custom reservation: [('floor', 'floor', 1, 0, 0)]
got reserved one from 0
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0)
sampled one from 0
try: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
sampled one from 0
try: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0)
sampled one from 0
CustomRule call
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
transition motion tried: True
Rule for AbstractObject
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
result: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = fail
sampled one from 0
CustomRule call
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: [1]
CustomRule call
Rule for AbstractObject
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = fail
('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) , diff: [

result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = fail
('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [1]
sampled one from 3
try: 2 - ('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [1]
CustomRule call
try: 3 - ('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
Rule for AbstractObject
Rule for AbstractObject
CustomRule call
('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [1]
CustomRule call
Rule for AbstractObject
transition motion tried: True
try homing
result: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = fail
('floor', 'grip1', 0, 0, 0)->('floor', 'floor', 0, 0, 0) , diff: [1]
make custom reservation: [<pkg.planning.scene.State instance at 0x7f818a576230>, ('floor', 'grip1', 1, 0, 0)]
('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) , diff: [1]
Rule for AbstractObject
result: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor

Reserve next wp from 7
try homing
Rule for AbstractObject
Removing 7 from [7] as next wp is reserved
try: 5 - ('floor', 'grip1', 1, 0, 0)->('floor', 'grip1', 2, 0, 0)
branching: 3->5 (0.94/100.0 s, steps/err: 12(39.6990776062 ms)/0.00164610391918)
sampled one from 9
make custom reservation: [<pkg.planning.scene.State instance at 0x7f818a4d5960>, ('grip1', 'floor', 1, 0, 0)]
try homing
sampled one from 0
make custom reservation: [<pkg.planning.scene.State instance at 0x7f818a576230>, ('floor', 'grip1', 1, 0, 0)]
make custom reservation: [('floor', 'grip1', 2, 0, 0)]
got reserved one from 6
result: 0 - ('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = success
transition motion tried: True
result: 4 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 1, 0, 0) = success
try: 0 - ('floor', 'floor', 0, 0, 0)->('floor', 'grip1', 0, 0, 0)
try: 9 - ('grip1', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
make custom reservation: [<pkg.planning.scene.State instance at 0x7f818a4d5960>, ('g

result: 6 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = success
make custom reservation: [('floor', 'grip1', 2, 0, 0)]
make custom reservation: [('floor', 'grip1', 1, 0, 0)]
result: 8 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 0, 0, 0) = success
result: 12 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 1, 0, 0) = success
got reserved one from 13
branching: 6->16 (1.32/100.0 s, steps/err: 34(247.437953949 ms)/4.71774765764e-16)
got reserved one from 14
try homing
try: 13 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0)
branching: 0->17 (1.34/100.0 s, steps/err: 29(251.266002655 ms)/0.001225694803)
branching: 8->20 (1.33/100.0 s, steps/err: 18(193.306922913 ms)/8.39341439038e-16)
try: 14 - ('floor', 'grip1', 1, 0, 0)->('floor', 'grip1', 2, 0, 0)
branching: 12->18 (1.33/100.0 s, steps/err: 12(40.8108234406 ms)/0.00177251316181)
got reserved one from 16
try: 16 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 1, 0, 0)
make custom reservation: [<pkg.plann

Rule for SweepLineTask
branching: 20->27 (1.69/100.0 s, steps/err: 12(97.0189571381 ms)/0.00190468297686)
('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: []
CustomRule call
try: 27 - ('floor', 'grip1', 1, 0, 0)->('floor', 'grip1', 2, 0, 0)
Reserve next wp from 28
make custom reservation: [('grip1', 'floor', 1, 0, 0)]
Removing 28 from [28] as next wp is reserved
sampled one from 11
try: 11 - ('grip1', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0)
joint motion tried: True
('floor', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) , diff: [0]
transition motion tried: True
CustomRule call
make custom reservation: [('floor', 'floor', 2, 0, 0)]
CustomRule call
result: 15 - ('grip1', 'floor', 0, 0, 0)->('grip1', 'floor', 0, 0, 0) = success
('floor', 'grip1', 1, 0, 0)->('floor', 'grip1', 2, 0, 0) , diff: [2]
transition motion tried: True
result: 23 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = success
Rule for AbstractObject
CustomRule call
CustomRule call
branch

...

make custom reservation: [('floor', 'floor', 2, 0, 0)]
CustomRule call
result: 324 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 0, 0, 0) = success
try: 338 - ('floor', 'floor', 1, 0, 0)->('floor', 'floor', 2, 0, 0)
CustomRule call
('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) , diff: [1]
branching: 324->341 (22.71/100.0 s, steps/err: 25(489.90893364 ms)/7.35712374733e-16)
Rule for AbstractObject
got reserved one from 341
result: 329 - ('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) = fail
try: 341 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0)
result: 335 - ('floor', 'floor', 0, 0, 0)->('floor', 'floor', 1, 0, 0) = success
('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) , diff: [1]
CustomRule call
Rule for AbstractObject
result: 329 - ('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) = fail
CustomRule call
transition motion tried: True
got reserved one from 340
branching: 335->340 (22.74/100.0 s, steps/err: 12(95.6399440765 ms)/0.000

CustomRule call
Reserve next wp from 345
('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) , diff: [1]
Rule for AbstractObject
Rule for AbstractObject
CustomRule call
Removing 345 from [344, 345] as next wp is reserved
Rule for AbstractObject
CustomRule call
make custom reservation: [('floor', 'grip1', 2, 0, 0)]
sampled one from 329
result: 339 - ('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 1, 0, 0) = success
('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) , diff: [1]
result: 329 - ('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) = fail
result: 337 - ('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) = fail
Reserve next wp from 344
Removing 344 from [344] as next wp is reserved
branching: 339->345 (23.1/100.0 s, steps/err: 12(321.141004562 ms)/0.0018843475121)
('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2) , diff: [1]
try: 329 - ('floor', 'grip1', 2, 2, 2)->('floor', 'floor', 2, 2, 2)
result: 329 - ('floor', 'grip1', 2, 2, 2)->('floor', 'floor

======================= terminated 0: first answer acquired ===============================
transition motion tried: False
('floor', 'floor', 2, 2, 2)->('floor', 'grip1', 2, 2, 2) , diff: [1]
CustomRule call
CustomRule call
======================= terminated 18: first answer acquired from other agent ===============================
Rule for AbstractObject
======================= terminated 7: first answer acquired from other agent ===============================
result: 347 - ('floor', 'floor', 2, 2, 2)->('floor', 'grip1', 2, 2, 2) = fail
('floor', 'floor', 2, 2, 2)->('grip1', 'floor', 2, 2, 2) , diff: [0]
('floor', 'floor', 2, 2, 2)->('grip1', 'floor', 2, 2, 2) , diff: [0]
CustomRule call
('floor', 'grip1', 0, 0, 0)->('floor', 'grip1', 1, 0, 0) , diff: [2]
Rule for AbstractObject
Rule for AbstractObject
Rule for SweepLineTask
result: 347 - ('floor', 'floor', 2, 2, 2)->('grip1', 'floor', 2, 2, 2) = fail
======================= terminated 15: first answer acquired from other agent =====

======================= terminated 19: first answer acquired from other agent ===============================
constrained motion tried: False
CustomRule call
('grip1', 'floor', 1, 0, 0)->('grip1', 'floor', 2, 0, 0) , diff: [2]
Rule for SweepLineTask
result: 315 - ('grip1', 'floor', 1, 0, 0)->('grip1', 'floor', 2, 0, 0) = fail
======================= terminated 14: first answer acquired from other agent ===============================
constrained motion tried: False
CustomRule call
('floor', 'grip1', 1, 0, 0)->('floor', 'grip1', 2, 0, 0) , diff: [2]
Rule for SweepLineTask
result: 344 - ('floor', 'grip1', 1, 0, 0)->('floor', 'grip1', 2, 0, 0) = fail
======================= terminated 6: first answer acquired from other agent ===============================


Process Process-6:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 153, in __search_loop
    display=display, dt_vis=dt_vis, **kwargs)
  File "pkg/planning/pipeline.py", line 200, in test_connection
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
  File "pkg/planning/motion/interface.py", line 105, in plan_transition
    redundancy_values=redundancy_values, **kwargs)
  File "pkg/planning/motion/moveit/moveit_planner.py", line 240, in plan_algorithm
    group_name, tool.geometry.link_name, goal_pose, target.geometry.link_name, tuple(from_Q), timeout=timeout)
  File "pkg/planning/motion/moveit/moveit_py.py", line 113, in plan_py
    JointState(self.joint_num, *Q_init), plannerconfig, timeout)
KeyboardInt

In [33]:
print(gtimer)

plan: 	23465.0 ms/1 = 23465.316 ms (23465.316/23465.316)



## MUST CHECK! no more than 1 trial from non-terminal nodes
## MUST CHECK! no more than 1 trial to same framed motion
## MUST CHECK! efficient binding/param sampling for gripper

In [ ]:
for k, v in tplan.custom_rule.call_count_dict.items():
    print("{}: {} - {}".format(k,tplan.snode_dict[k].state.node, sorted(v)))

## play searched plan

In [36]:
ppline.play_schedule(snode_schedule[:2], period=0.001)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [37]:
snode_schedule = ppline.add_return_motion(snode_schedule)

joint motion tried: True


In [38]:
ppline.play_schedule(snode_schedule, period=0.03)

In [ ]:
print("schedule length: {}".format(len(snode_schedule)))
for i_s,  snode in enumerate(snode_schedule):
    print("{}: {}".format(i_s, snode.state.node))

## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정
* goal-directed extension 추가.

In [ ]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

## extend preserving goal-matching items

In [ ]:
print(gtimer)

## extend only no reservation

In [ ]:
print(gtimer)

## no extension

In [ ]:
print(gtimer)

## extend_toward goal

In [ ]:
print(gtimer)